# Moment of Inertia Tensor (discrete) - SKP

The angular momentum is
$$ L = I \omega $$
The Moment of Inertia tensor is;
$$ I =\begin{bmatrix}
I_{xx} & I_{xy} & I_{xz} \\
I_{yx} & I_{yy} & I_{yz} \\
I_{zx} & I_{zy} & I_{zz} \\
\end{bmatrix}$$
where,
$$ I_{xx} = \sum_i m_i (r_i^2 - x_i^2)$$
$$ I_{yy} = \sum_i m_i (r_i^2 - y_i^2)$$
$$ I_{zz} = \sum_i m_i (r_i^2 - z_i^2)$$
$$ I_{xy} = I_{yx} = -\sum_i m_ix_iy_i $$
$$ I_{yz} = I_{zy} = -\sum_i m_iy_iz_i $$
$$ I_{zx} = I_{xz} = -\sum_i m_iz_ix_i $$
$$(r_i = \sqrt{x_i^2 + y_i^2 + z_i^2})$$

By using this concept, we can calculate MOI of discrete mass system very easily. To calculate this for continuous mass (rod, disc etc.), the challenging task is to create a discrete distribution that can be able to represent a continuous mass distribution.

Here, moment of inretia is calculated for
* Discrete masses
* Linear mass (rod)
* Annular disc
* Disc

In [1]:
import numpy as np
import numpy.linalg as nlinalg
import scipy.linalg as slinalg
import plotly.graph_objects as go

Defined functions:
* **`moi_tensor`**
* **`moi_tensor_diagonalized`**
* **`moi_wrt_axis`**
* **`plot_mass_distribution`**

In [2]:
def moi_tensor(mass_distribution, mass):
    Ixxl, Iyyl, Izzl = [], [], []
    Ixyl, Iyzl, Izxl = [], [], []
    for i in range(len(mass)):
        mi = mass[i]
        xi = mass_distribution[i, 0]
        yi = mass_distribution[i, 1]
        zi = mass_distribution[i, 2]
        ri = np.sqrt(xi**2+yi**2+zi**2)
        Ixxi = mi*(ri**2-xi**2)
        Iyyi = mi*(ri**2-yi**2)
        Izzi = mi*(ri**2-zi**2)
        Ixxl.append(Ixxi), Iyyl.append(Iyyi), Izzl.append(Izzi)
        Ixyi, Iyzi, Izxi = -mi*xi*yi, -mi*yi*zi, -mi*zi*xi
        Ixyl.append(Ixyi), Iyzl.append(Iyzi), Izxl.append(Izxi)
    Ixx, Iyy, Izz = np.sum(Ixxl), np.sum(Iyyl), np.sum(Izzl)
    Ixy, Iyz, Izx = np.sum(Ixyl), np.sum(Iyzl), np.sum(Izxl)
    Iyx, Izy, Ixz = Ixy, Iyz, Izx
    Imat = np.array([[Ixx, Ixy, Ixz], [Iyx, Iyy, Iyz], [Izx, Izy, Izz]])
    return Imat

def moi_tensor_diagonalized(moi_tensor):
    eigvals, Pmat = slinalg.eig(moi_tensor)
    D1 = slinalg.inv(Pmat).dot(moi_tensor@Pmat)
    return D1

def moi_wrt_axis(the_axis, moi_tensor):
    A = the_axis/nlinalg.norm(the_axis)
    return A.dot(moi_tensor.dot(A.T))

def plot_mass_distribution(mass_distribution):
    points = mass_distribution
    x = [point[0] for point in points]
    y = [point[1] for point in points]
    z = [point[2] for point in points]

    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2))])

    fig.update_layout(scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title='z'))
    fig.show()

## Discrete masses

In [3]:
distr = 0.5*np.array([[-1,-1,0],[1,-1,0],[1,1,0],[-1,1,0]])  # INPUT
ms = np.array([1,2,3,4])        # INPUT
plot_mass_distribution(distr)

I1 = moi_tensor(distr, ms)
display('moi tensor', I1)
eigvals, Pmat = slinalg.eig(I1)
for j in range(len(eigvals)):
    li = eigvals[j]
    vi = Pmat[:, j]
    print(f'eigenvalue={li:4f}; eigenvector={vi}')

D1 = slinalg.inv(Pmat).dot(I1@Pmat)
display('diagonalised moi tensor', D1, moi_tensor_diagonalized(I1))

ax = np.array([1, 1, 2])
print(f'moi wrt axis {ax} is {moi_wrt_axis(ax, I1)}')

'moi tensor'

array([[2.5, 0.5, 0. ],
       [0.5, 2.5, 0. ],
       [0. , 0. , 5. ]])

eigenvalue=3.000000+0.000000j; eigenvector=[0.70710678 0.70710678 0.        ]
eigenvalue=2.000000+0.000000j; eigenvector=[-0.70710678  0.70710678  0.        ]
eigenvalue=5.000000+0.000000j; eigenvector=[0. 0. 1.]


'diagonalised moi tensor'

array([[3., 0., 0.],
       [0., 2., 0.],
       [0., 0., 5.]])

array([[3., 0., 0.],
       [0., 2., 0.],
       [0., 0., 5.]])

moi wrt axis [1 1 2] is 4.333333333333336


## Linear mass (Rod)

In [4]:
M = 1
L = 5   # INPUT
x = np.linspace(-L/2,L/2,1000)
y = np.zeros(x.shape)
z = np.zeros(x.shape)
distr = np.array([x, y, z]).T
m = M/x.size
ms = m*np.ones(x.shape)
plot_mass_distribution(distr)

I1 = moi_tensor(distr, ms)
display('moi_tensor', I1, 'exact value: Iz=ML2/12', M*L**2/12)


'moi_tensor'

array([[0.        , 0.        , 0.        ],
       [0.        , 2.08750417, 0.        ],
       [0.        , 0.        , 2.08750417]])

'exact value: Iz=ML2/12'

2.0833333333333335

## Ring

In [5]:
import numpy as np

R = 5.0  # INPUT - Radius of the ring
num_points = 1000 # INPUT

z_coordinate = 0.0
theta = np.linspace(0, 2 * np.pi, num_points)
x = R * np.cos(theta)
y = R * np.sin(theta)
z = np.full(num_points, z_coordinate)

distr = np.column_stack((x, y, z))
M = 1
m = M/num_points
ms = m*np.ones((num_points,))
plot_mass_distribution(distr)
I1 = moi_tensor(distr, ms)
display('moi_tensor', I1, 'exact value: Iz=MR2', M*R**2)

'moi_tensor'

array([[1.24875000e+01, 2.22044605e-16, 0.00000000e+00],
       [2.22044605e-16, 1.25125000e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.50000000e+01]])

'exact value: Iz=MR2'

25.0

## Annular disc
$$ I = \frac{1}{2}\frac{M}{R_2^2 - R_1^2} \left(R_2^4 - R_1^4\right) $$


In [6]:
import numpy as np

# Define the parameters of the annular disc
R1 = 2.0  # Inner radius of the annulus
R2 = 4.0  # Outer radius of the annulus
num_points = 10000
z_coordinate = 0.0  # Elevation of the annular disc in the Z axis

# Generate uniformly distributed points within the annular disc
theta = 2 * np.pi * np.random.rand(num_points)
r = np.sqrt(np.random.uniform(R1**2, R2**2, num_points))

# Calculate the (x, y) coordinates of the points
x = r * np.cos(theta)
y = r * np.sin(theta)

# Create the Z coordinate for all points (elevation)
z = np.full(num_points, z_coordinate)

# Create a 2D NumPy array of the coordinates [x, y, z]
distr = np.column_stack((x, y, z))
M = 1
m = M/num_points
ms = m*np.ones((num_points,))
plot_mass_distribution(distr)

I_exact = 0.5*M/(R2**2-R1**2) * (R2**4-R1**4)
display('moi_tensor', I1, 'exact value of Iz', I_exact)

'moi_tensor'

array([[1.24875000e+01, 2.22044605e-16, 0.00000000e+00],
       [2.22044605e-16, 1.25125000e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.50000000e+01]])

'exact value of Iz'

10.0

## Disc

In [7]:
import numpy as np

# Define the parameters of the disc
R = 1  # Adjust the radius as needed
num_points = 10000
z_coordinate = 0.0  # Elevation of the disc in the Z axis

# Generate uniformly distributed points within the disc
theta = 2 * np.pi * np.random.rand(num_points)
r = R * np.sqrt(np.random.rand(num_points))

# Calculate the (x, y) coordinates of the points
x = r * np.cos(theta)
y = r * np.sin(theta)

# Create the Z coordinate for all points (elevation)
z = np.full(num_points, z_coordinate)

# Create a 2D NumPy array of the coordinates [x, y, z]
distr = np.column_stack((x, y, z))
M = 1
m = M/num_points
ms = m*np.ones((num_points,))
plot_mass_distribution(distr)


I1 = moi_tensor(distr, ms)
display('moi_tensor', I1, 'exact value: Iz=MR2/2', M*R**2/2)

'moi_tensor'

array([[ 0.24796206, -0.00060629,  0.        ],
       [-0.00060629,  0.24814664,  0.        ],
       [ 0.        ,  0.        ,  0.4961087 ]])

'exact value: Iz=MR2/2'

0.5